# Code for gathering gathering dataset
First cell) Here our MP4s will be converted to a large swath of images for labeling. 

In [ ]:
import cv2
import os

def extract_frames_from_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get a list of all MP4 files in the input folder
    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
    if not video_files:
        print("No MP4 files found in the input folder.")
        return
    
    print(f"Found {len(video_files)} MP4 files to process.")

    for video_file in video_files:
        video_path = os.path.join(input_folder, video_file)
        video_name = os.path.splitext(video_file)[0]
        
        # Create a subfolder for frames from this video
        video_output_folder = os.path.join(output_folder, video_name)
        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)
        
        # Open the video file
        video = cv2.VideoCapture(video_path)
        if not video.isOpened():
            print(f"Error: Could not open video {video_file}. Skipping...")
            continue
        
        # Extract frames
        frame_count = 0
        success, frame = video.read()
        print(f"Processing {video_file}...")
        
        while success:
            frame_filename = os.path.join(video_output_folder, f"frame_{frame_count:05d}.jpg")
            cv2.imwrite(frame_filename, frame)
            frame_count += 1
            
            # Read next frame
            success, frame = video.read()
            
            if frame_count % 100 == 0:
                print(f"Extracted {frame_count} frames from {video_file}...")
        
        video.release()
        print(f"Finished extracting {frame_count} frames from {video_file}.")

    print(f"All videos processed. Frames saved to {output_folder}.")

input_folder = "/Users/jakeziegler/Desktop/PROJECTS/SPRAGUE EYE/JSE TRAINING" 
output_folder = "frames_output" 
extract_frames_from_folder(input_folder, output_folder)

# Creating a training and testing set for images of behaviors
This will randomly sample from a series of direcrtories and create a new df, which then is used to sort and create a new folder

In [ ]:
import pandas as pd
import os
import shutil

def create_image_dataframe(root_directory):
    directories = []
    image_paths = []

    for subdir, _, files in os.walk(root_directory):
        for file in files:
            # Only process image files
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                full_path = os.path.join(subdir, file)
                directories.append(subdir.split("/")[-1])
                image_paths.append(full_path)
    
    df = pd.DataFrame({
        'Directory': directories,
        'Image Path': image_paths
    })
    return df

root_dir = "/Users/jakeziegler/Desktop/PROJECTS/SPRAGUE EYE/JSE TRAINING/FRAMES" 
image_df = create_image_dataframe(root_dir)
sample_df = image_df.sample(n=500, random_state=42)

file_paths = sample_df['Image Path']

# Destination folder
destination_folder = "/Users/jakeziegler/Desktop/PROJECTS/SPRAGUE EYE/JSE TRAINING/SAMPLED FRAMES"
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Loop through the Series and copy each file
for file_path in file_paths:
    if os.path.exists(file_path):
        shutil.copy(file_path, destination_folder)
        print(f"Copied: {file_path} to {destination_folder}")
    else:
        print(f"File not found: {file_path}")

# Creating a training and testing set for polygons of cages
Takes the COCO dataset and splits into training and testing for training down the road

# Creating more images for training data
Takes a folder and flips images on vertical and horizontal axes

In [ ]:
import os
from PIL import Image

def flip_images(input_folder, output_folder, flip_type='horizontal'):
    """
    Flips images in the input folder and saves them in the output folder.

    :param input_folder: Path to the folder containing images to flip.
    :param output_folder: Path to the folder where flipped images will be saved.
    :param flip_type: Type of flip ('horizontal' or 'vertical').
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            try:
                with Image.open(input_path) as img:
                    if flip_type == 'horizontal':
                        flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
                    elif flip_type == 'vertical':
                        flipped_img = img.transpose(Image.FLIP_TOP_BOTTOM)
                    else:
                        print(f"Invalid flip_type: {flip_type}. Skipping {filename}.")
                        continue

                    flipped_img.save(output_path)
                    print(f"Flipped and saved: {output_path}")
            except Exception as e:
                print(f"Failed to process {filename}: {e}")

if __name__ == "__main__":
    flip_images(input_folder, output_folder, flip_type)
    flip_images()

# Folder content combiner
Combines content of folders

In [ ]:
import os
import shutil

def combine_folders(source_folders, destination_folder):
    """
    Combines the contents of multiple folders into a single destination folder.

    Args:
        source_folders (list of str): Paths to the source folders.
        destination_folder (str): Path to the destination folder.
    """
    # Ensure the destination folder exists
    os.makedirs(destination_folder, exist_ok=True)

    for folder in source_folders:
        if not os.path.exists(folder):
            print(f"Source folder does not exist: {folder}")
            continue

        for root, dirs, files in os.walk(folder):
            # Determine the relative path to recreate the folder structure
            relative_path = os.path.relpath(root, folder)
            dest_path = os.path.join(destination_folder, relative_path)

            # Create directories in the destination
            os.makedirs(dest_path, exist_ok=True)

            # Copy each file
            for file in files:
                src_file = os.path.join(root, file)
                dest_file = os.path.join(dest_path, file)

                # If the file already exists in the destination, add a numeric suffix to avoid overwriting
                if os.path.exists(dest_file):
                    base, ext = os.path.splitext(file)
                    counter = 1
                    while os.path.exists(dest_file):
                        dest_file = os.path.join(dest_path, f"{base}({counter}){ext}")
                        counter += 1

                shutil.copy2(src_file, dest_file)
                print(f"Copied: {src_file} -> {dest_file}")

# Example usage
source_folders = []
destination_folder = "combined_folder"
combine_folders(source_folders, destination_folder)


In [1]:
import cv2
import os

def images_to_video(image_folder, output_file, fps):
    """
    Convert a folder of images into a video.

    Parameters:
    - image_folder: Path to the folder containing images.
    - output_file: Path to the output video file (e.g., 'output.mp4').
    - fps: Frames per second for the output video.
    """
    # Get a sorted list of image files in the folder
    images = [img for img in os.listdir(image_folder) if img.endswith((".png", ".jpg", ".jpeg"))]
    images.sort()  # Ensure images are in the correct order

    if not images:
        print("No images found in the folder.")
        return

    # Read the first image to get dimensions
    first_image_path = os.path.join(image_folder, images[0])
    frame = cv2.imread(first_image_path)
    height, width, _ = frame.shape

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # For .mp4 output
    video = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    # Loop through the images and write them to the video
    for image in images:
        image_path = os.path.join(image_folder, image)
        frame = cv2.imread(image_path)
        video.write(frame)

    # Release the video writer
    video.release()
    print(f"Video saved as {output_file}")

# Example usage
if __name__ == "__main__":
    image_folder = "/Users/jakeziegler/Desktop/PROJECTS/SPRAGUE-EYE/JSE-TRAINING/FRAMES/JSE2"
    output_file = "/Users/jakeziegler/Desktop/untitled folder/output.mp4"
    fps = 30  # Adjust frames per second as needed

    images_to_video(image_folder, output_file, fps)


Video saved as /Users/jakeziegler/Desktop/untitled folder/output.mp4


# Video Flipper

In [ ]:
import cv2
import os

def flip_video(input_path, output_path, flip_code):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error opening video file: {input_path}")
        return

    # Get video properties
    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        flipped_frame = cv2.flip(frame, flip_code)
        out.write(flipped_frame)
    
    cap.release()
    out.release()
    print(f"Saved flipped video: {output_path}")

def process_videos_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(('.mp4', '.avi', '.mov', '.mkv')):
            input_path = os.path.join(folder_path, filename)
            name, ext = os.path.splitext(filename)
            
            # Flip horizontally
            flip_video(input_path, os.path.join(folder_path, f"{name}_flippedH{ext}"), 1)
            # Flip vertically
            flip_video(input_path, os.path.join(folder_path, f"{name}_flippedV{ext}"), 0)
            # Flip both ways
            flip_video(input_path, os.path.join(folder_path, f"{name}_flippedHV{ext}"), -1)

folder_path = input("Enter the folder path containing videos: ")
process_videos_in_folder(folder_path)
print("Processing complete.")


# CSV Checks

In [4]:
import pandas as pd
csv_path = "/Users/jakeziegler/Desktop/MyNewFolder/frames_JSE1DLC_Resnet50_social_expJan31shuffle1_snapshot_010_el.csv"

df = pd.read_csv(csv_path)
print(df.shape)
df.head()



(14561, 43)


,scorer,DLC_Resnet50_social_expJan31shuffle1_snapshot_010,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.1,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.2,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.3,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.4,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.5,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.6,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.7,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.8,...,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.32,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.33,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.34,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.35,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.36,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.37,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.38,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.39,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.40,DLC_Resnet50_social_expJan31shuffle1_snapshot_010.41
0,individuals,adult,adult,adult,adult,adult,adult,adult,adult,adult,...,juvie,juvie,juvie,juvie,juvie,juvie,juvie,juvie,juvie,juvie
1,bodyparts,nose,nose,nose,left_eye,left_eye,left_eye,right_eye,right_eye,right_eye,...,left_ear,right_ear,right_ear,right_ear,mid_back,mid_back,mid_back,tail_base,tail_base,tail_base
2,coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,...,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood
3,0,1350.781982421875,491.5199890136719,0.5839999914169312,1332.0989990234375,514.43798828125,0.8899999856948853,1357.6199951171875,530.093017578125,0.6819999814033508,...,0.9390000104904175,1320.56396484375,686.3569946289062,0.9750000238418579,NaN,NaN,0.0,NaN,NaN,0.0
4,1,1386.031982421875,539.5960083007812,0.39399999380111694,1336.176025390625,515.6500244140625,0.5540000200271606,1362.8050537109375,534.9089965820312,0.722000002861023,...,0.9750000238418579,1321.2039794921875,687.3270263671875,0.9860000014305115,NaN,NaN,0.0,NaN,NaN,0.0


In [7]:
adult_exists = df.iloc[3:, [1, 2, 4, 5, 7, 8]].notna().all(axis=1)
adult_exists

3        True
4        True
5        True
6        True
7        True
         ... 
14556    True
14557    True
14558    True
14559    True
14560    True
Length: 14558, dtype: bool